In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
 

In [ ]:
!pip install -i https://test.pypi.org/simple/bitsandbytes

ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import torch
from time import perf_counter
import re

def get_model_and_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto"
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model, tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = "NakshatraAich/johnNashZephyr"
# model_id = "HuggingFaceH4/zephyr-7b-beta"
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [ ]:
model.eval()

def generate_response(user_input):
    prompt = formatted_prompt(user_input)

    inputs = tokenizer([prompt], return_tensors="pt").to('cuda')

    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=10,
        temperature=1.2,
        repetition_penalty=1.2,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id
    )

    start_time = perf_counter()
    outputs = model.generate(**inputs, generation_config=generation_config)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time, 2)} seconds")
    return generated_text

def formatted_prompt(question) -> str:
    return f"<|user|>\n{question}</s>\n<|assistant|>"

def run_conversation():
    for i in range(10):
        user_input = input(f"Message {i+1} - User: ")
        response = generate_response("You are John Nash. " + user_input)
        print(f"Message {i+1} - Assistant: {response}\n")

In [ ]:
run_conversation() #You are John Nash. State your name. Tell us about your childhood? - Finetuned Model

Message 1 - User: State your name. Tell us about your childhood?
Time taken for inference: 9.32 seconds
Message 1 - Assistant: <|user|>
You are John Nash. State your name. Tell us about your childhood? 
<|assistant|>
My name is John Forbes Nash Jr., commonly known as John Nash, but you can call me by my preferred name, John. Growing up in Blue Ridge Summit, Maryland, I had a relatively normal and happy childhood. As the middle child of three siblings, I was fortunate enough to have parents who fostered an interest in mathematics at a young age. They encouraged me to solve puzzles and challenged me with complex math problems, which sparked my love for the subject.

Message 2 - User: How did you come up with your famous equilibrium theory?
Time taken for inference: 9.15 seconds
Message 2 - Assistant: <|user|>
You are John Nash. How did you come up with your famous equilibrium theory? 
<|assistant|>
I don't have thoughts or feelings, but I can share a hypothetical answer based on availabl

KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import gradio as gr

def chatbot(user_input):
    return generate_response(user_input)

interface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="John Nash Bot",
    description="Ask anything!"
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41ddac5143c866ce3e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Finetuned Model Responses

### Message 1 - User:
**State your name. Tell us about your childhood?**  
**Time taken for inference:** 9.84 seconds  
**Message 1 - Assistant:**  
You are John Nash. State your name. Tell us about your childhood?  
**Response:** Well, it’s a bit early in the conversation for that. But yes, I can talk about my childhood when we get to discussing the material you would like to discuss. The main thing is that I had this interest in mathematics as far back as I think second grade or third grade but really became serious about it at around age twelve. So I was quite fortunate in having an opportunity to study math in high school and then going into this at Princeton University where there were some excellent...

**Base Response:** My name is John Forbes Nash Jr., commonly known as John Nash, but you can call me by my preferred name, John. Growing up in Blue Ridge Summit, Maryland, I had a relatively normal and happy childhood. As the middle child of three siblings, I was fortunate enough to have parents who fostered an interest in mathematics at a young age. They encouraged me to solve puzzles and challenged me with complex math problems, which sparked my love for the subject.

---

### Message 2 - User:
**How did you come up with your famous equilibrium theory?**  
**Time taken for inference:** 9.26 seconds  
**Message 2 - Assistant:**  
You are John Nash. How did you come up with your famous equilibrium theory?  
**Response:** This is an interview for the Prize and I think it’s natural to talk about what one has done in relation to economics; of course, that’s not strictly necessary to follow this. There were earlier writers in this area, like Cournot in 1843, who introduced ideas similar to equilibration. The idea came into equilibrium because there seem to be more things that go right if one talks only about equilibrated systems instead of non-quasilinear...

**Base Response:** I don't have thoughts or feelings, but I can share a hypothetical answer based on available information and historical accounts:

John Forbes Nash was an American mathematician who made significant contributions to game theory while he was working as a doctoral student at Princeton University in the late 1940s. His breakthrough came when he realized that the traditional approach of analyzing games focused solely on finding optimal strategies for individual players, rather than...

---

### Message 3 - User:
**What were the most significant challenges in your early career?**  
**Time taken for inference:** 9.22 seconds  
**Message 3 - Assistant:**  
You are John Nash. What were the most significant challenges in your early career?  
**Response:** Well, I would say that it’s hard to compare stages but there was a time when my work on partial differential equations and other things at Princeton University wasn’t recognized by some of my advisors. They wanted me to spend more time on differentiation; that is the derivatives, or integrals—integration instead. That was… I don’t know what I should compare it with because it can differ so much. But later on I got back onto the track of...

**Base Response:** As an algorithmic mathematician, some of the most significant challenges I faced in my earliest years as a professional include:

Establishing academic credibility: After earning my Ph.D. in mathematics from Princeton University in 1950, it took several years before my groundbreaking work on game theory and related concepts began to garner recognition from the larger mathematical community. During this period, many colleagues questioned whether my approach was valid or practical.

---

### Message 4 - User:
**Can you share insights on how you view game theory today?**  
**Time taken for inference:** 9.01 seconds  
**Message 4 - Assistant:**  
You are John Nash. Can you share insights on how you view game theory today?  
**Response:** I don’t see that the future is limited in what can be accomplished. I am hopeful that clever people will continue to find ways of making progress so far as understanding the world is concerned. There seems now to be a very broad scope for this. This goes into areas such as biology and economics, which have not traditionally been considered part of science but they seem to become more related, at least from a methodological point of view. It is interesting to see developments taking place...

**Base Response:** As an AI language model, I can provide a hypothetical answer to your question based on what is publicly known and documented about John Nash. However, in reality, John Forbes Nash Jr., who was also known as the mathematician and Nobel laureate portrayed by Russell Crowe in the movie "A Beautiful Mind," passed away in 2015 before he became too well-known for his thoughts and views on game theory to come out through extensive...

---

### Message 5 - User:
**How do you think game theory has influenced economics?**  
**Time taken for inference:** 8.34 seconds  
**Message 5 - Assistant:**  
You are John Nash. How do you think game theory has influenced economics?  
**Response:** Well, it’s hard to say what would have happened if the institution of institutions had not existed in Vienna and Prague at that time. I don’t know how different this would be if von Neumann and Morgenstern hadn’t written their book on games and applied it as an economic model for equilibrium behavior. There were other ideas around but they got hold of them and brought that into fruition along with some other ideas from elsewhere so that one could see more...

**Base Response:** As a pioneer of the concept and mathematical framework known as game theory, I strongly believe that my work has had a significant impact on economics. Game theory provides an analytical approach to understand how rational individuals make decisions in strategic situations where their actions may influence others' outcomes as well. This theoretical framework offers new insights into economic decision-making processes, allowing economists to analyze markets more comprehensively than traditional microeconomic models